In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

video = cv2.VideoCapture("003.mp4")  # Load video file

# Initialize parameters for Lucas-Kanade method
lk_params = dict(winSize=(15, 15),
                 maxLevel=4,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Read the first frame
success, prev_frame = video.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Define initial bounding box for the tracker
bbox = cv2.selectROI("Frame", prev_frame, False)
x, y, w, h = bbox

# Initialize tracker with the initial bounding box
tracker = cv2.TrackerCSRT_create()
tracker.init(prev_frame, bbox)

while True:
    # Read a new frame
    success, frame = video.read()
    if not success:
        break

    # Convert frame to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray,
                                           np.array([[x, y]]).astype(np.float32),
                                           None, **lk_params)

    # Select good points
    good_new = p1[st == 1]
    good_old = np.array([[x, y]]).astype(np.float32)

    # Calculate bounding box from the tracked points
    if len(good_new) > 0:
        x, y = good_new.ravel()
        x, y = int(x), int(y)

    # Update the bounding box
    bbox = (x, y, w, h)

    # Initialize tracker with the updated bounding box
    tracker.init(frame, bbox)

    # Update the previous frame and previous points
    prev_gray = frame_gray.copy()

    # Draw bounding box on the object
    (x, y, w, h) = [int(i) for i in bbox]
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the frame with the bounding box
    cv2_imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()



